In [1]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
import pandas as pd
import numpy as np

In [3]:
import re
import stanza
stanza.download('en')
nlp = stanza.Pipeline('en', processors = "tokenize,mwt,pos,lemma,depparse") 

2022-01-09 22:46:28 INFO: Downloading default packages for language: en (English)...
2022-01-09 22:46:30 INFO: File exists: /Users/guilhermealbm/stanza_resources/en/default.zip.
2022-01-09 22:46:33 INFO: Finished downloading models and saved to /Users/guilhermealbm/stanza_resources.
2022-01-09 22:46:33 WARNING: Can not find mwt: default from official model list. Ignoring it.
2022-01-09 22:46:33 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2022-01-09 22:46:33 INFO: Use device: cpu
2022-01-09 22:46:33 INFO: Loading: tokenize
2022-01-09 22:46:33 INFO: Loading: pos
2022-01-09 22:46:33 INFO: Loading: lemma
2022-01-09 22:46:33 INFO: Loading: depparse
2022-01-09 22:46:33 INFO: Done loading processors!


In [4]:
df = pd.read_csv("preprocessing/final_file.csv")

In [5]:
df.shape[0]

1048575

In [6]:
df_with_wiki = pd.read_csv("preprocessing/tags_with_wiki.csv")

In [7]:
start_shape = df_with_wiki.shape[0]
start_shape

44447

In [8]:
df_with_wiki = df_with_wiki.dropna()
df_with_wiki.shape[0]

41589

### NLP process

In [9]:
df_with_wiki["root"] = ""

for index, row in df_with_wiki.iterrows():
    text = row["Body"].lower().replace(",", "").replace("\n", " ")
    
    doc = nlp(text)
    
    definition_set = False
    for sentence in doc.sentences:        
        nsubj, cop, root = False, False, False
        sent_dict = sentence.to_dict()
        for word in sent_dict:
            if word['deprel'] == "nsubj" and word['xpos'][0] == 'N':
                nsubj = True
            elif nsubj and word['deprel'] == "cop":
                cop = True
                root = True
                df_with_wiki.at[index, 'root'] = sent_dict[word['head'] - 1]['text']

        if nsubj and cop and root:
            df_with_wiki.at[index, 'Body'] = str(sentence.text)
            definition_set = True
            break
        
    if index % 4000 == 0:
        print(str(index) + " / " + str(start_shape))


0 / 44447
4000 / 44447
8000 / 44447
12000 / 44447
16000 / 44447
20000 / 44447
24000 / 44447
28000 / 44447
32000 / 44447
36000 / 44447


In [10]:
df_with_wiki.shape[0]

41589

In [11]:
df_with_wiki.to_csv("tags_with_wiki_and_category.csv", index=False)

### Clean relationship df

In [12]:
df = df[df.source.isin(df_with_wiki.TagName)]
df = df[df.target.isin(df_with_wiki.TagName)]

In [13]:
df.shape[0]

936606

In [14]:
df.to_csv("tags_with_wiki_relationship.csv", index=False)